## Import Packages

In [1]:
from torch import *
from torchvision import *
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

## Load Pre-Trained Model

In [2]:
alex = models.alexnet(pretrained=True)
alex = alex.cuda()
alex.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Create Class and Register Hooks

In [3]:
class BackProp():
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.hook_layers()

    def hook_layers(self):
        def hook_function(module,grad_in, grad_out):
            self.gradients = grad_in[0]

        # Register hook to the first layer
        first_layer = list(self.model.features.modules())[0][0]
        print(first_layer)
        first_layer.register_backward_hook(hook_function)

    def get_gradients(self,image_tensor):
        image_tensor = image_tensor.unsqueeze(0).cuda()
        image_tensor.requires_grad_()
        self.model.zero_grad()

        output = self.model(image_tensor)
        
        one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().cuda()
        one_hot_output[0][56] = 1
        
        output.backward(gradient=one_hot_output)

        gradients_arr = self.gradients.data.cpu()
        
        return gradients_arr

In [4]:
def get_positive_negative_saliency(gradient):
    pos_saliency = (np.maximum(0, gradient) / gradient.max())
    neg_saliency = (np.maximum(0, -gradient) / -gradient.min())
    return pos_saliency, neg_saliency

## Load Image

In [5]:
norm = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

img_path = './images/snake.jpg'
image = Image.open(img_path)
image_tensor = norm(image)
print(image_tensor.size())
backprop = BackProp(alex)
output_gradients = backprop.get_gradients(image_tensor)

torch.Size([3, 224, 224])
Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))


In [6]:
output_gradients.shape

torch.Size([1, 3, 224, 224])

In [7]:
output_gradients = output_gradients.squeeze(0)
output_gradients.size()

torch.Size([3, 224, 224])

In [8]:
pos_saliency,neg_saliency = get_positive_negative_saliency(output_gradients)

In [9]:
permuted = output_gradients.permute(1,2,0).numpy()
permuted_pos = pos_saliency.permute(1,2,0).numpy()
permuted_neg = neg_saliency.permute(1,2,0).numpy()

In [10]:
im = Image.fromarray(np.uint8((permuted)*255))
im_pos = Image.fromarray(np.uint8((permuted_pos)*255))
im_neg = Image.fromarray(np.uint8((permuted_neg)*255))

In [11]:
im.save('/home/harsh/visualize-convnets/images/snake_backprop.jpg')

In [12]:
im_pos.save('/home/harsh/visualize-convnets/images/snake_backprop_pos.jpg')

In [13]:
im_neg.save('/home/harsh/visualize-convnets/images/snake_backprop_neg.jpg')